In [ ]:
# Python modules
import itertools
import time

# Data management
import numpy as np
import pandas as pd

# Plotting
import matplotlib.pylab as plt
import seaborn as sns
sns.set(style="ticks", font="Source Sans Pro", font_scale=.9, palette="muted")

# Fitting
import scipy.optimize as opt
from scipy.interpolate import interp1d

# Project modules
import luescher_nd.utilities as ut
import luescher_nd.zeta as zeta

In [ ]:
%load_ext blackcellmagic

In [ ]:
S3 = pd.read_csv("S3.dat", sep="\s+", header=None, names=["x", "y"])

divergencies = [-100]

for x, y, z in itertools.product(*[list(range(5))]*3):
    divergencies.append(x**2 + y**2 + z**2)
    
divergencies = np.unique(divergencies)

zeros = []
interpolators = {}
for start, end in zip(divergencies[:-1], divergencies[1:]):
    start += 0.001
    end -= 0.001
    
    x, y = S3[(S3.x > start) & (S3.x < end)].values.T
    
    if len(x) == 0:
        continue
    
    f = interp1d(x, y, kind="cubic")
    interpolators[(start, end)] = f
    
    res = opt.minimize(
        lambda x: f(x)**2, 
        np.array([(start+end)*1./2, ]), 
        tol=1.e-12,
        bounds=((x.min()+0.001, x.max()-0.001),)
    )
    zeros.append(res.x)
    
print(zeros)

In [ ]:
fig_zeta_3d, axs = plt.subplots(dpi=300, figsize=(4,3), ncols=1, sharey=True, sharex=True)

for ax in [axs]:
    ax.axhline(0, ls="-", c="grey", lw=.5, alpha=.5)

    for div in divergencies:
        if div < S3.x.max():
            ax.axvline(div, ls="--", c="grey", lw=.5, alpha=.5)


    S3.plot(ax=ax, x='x', y='y', lw=.2, ls="--", marker=".", ms=.5, label="$S_3(x)$")

    for zero in zeros:
        label = "$S_3(x_0) = 0$" if zero == zeros[0] else None
        if zero < S3.x.max():
            ax.scatter(zero, 0, marker="x", c="black", lw=.5, s=5, zorder=10, label=label)

    ax.set_ylim(-.5, .5)
    ax.legend(loc="best", fontsize="xx-small")

    ax.set_xlabel("$x$")
    ax.set_ylabel("$S_3(x)$")

    ax.set_xlim(S3.x.min(), S3.x.max())
    
plt.tight_layout()
plt.show(fig_zeta_3d)



## Fitting

Fit $c_0(a)$ such that $E = E(x_0, L)$, where

$$x_0 = \left( \frac{\gamma L}{2 \pi} \right)^2 = \frac{\mu E L^2}{2\pi^2}$$

In [ ]:
HBARC = 197.326 # MeV / fm
M_NUCLEON = (938.27 + 939.57) / 2 / HBARC # in fm^{-1}

mu = M_NUCLEON / 2
L = 1.
E0 = zeros[0] * 2 * np.pi**2 / mu / L**2

E0

In [ ]:
def get_laplace_coefficients(n_step: int):
    """Implementation fo finite step laplace derivative coefficients.

    See also https://en.wikipedia.org/wiki/Finite_difference_coefficient.

    Arguments
    ---------
        n_step: int
            Step range of derivative. Must be larger than 0.
    """
    out = {}

    if n_step < 1:
        raise ValueError("'n_step' must be larger than zero.")

    elif n_step == 1:
        out[0] = -2
        out[1] = out[-1] = 1

    elif n_step == 2:
        out[0] = -5 / 2
        out[1] = out[-1] = 4 / 3
        out[2] = out[-2] = -1 / 12

    elif n_step == 3:
        out[0] = -49 / 18
        out[1] = out[-1] = 3 / 2
        out[2] = out[-2] = -3 / 20
        out[3] = out[-3] = 1 / 90

    elif n_step == 4:
        out[0] = -205 / 72
        out[1] = out[-1] = 8 / 5
        out[2] = out[-2] = -1 / 5
        out[3] = out[-3] = 8 / 315
        out[4] = out[-4] = -1 / 560

    else:
        raise NotImplementedError("'n_step' not implemented for values larger than 6.")

    return out

In [ ]:
epsilons = [0.1, 0.08, 0.05]#, 0.02, 0.01]

data = []

for n_step in range(1, 5):
    print(n_step)
    for epsilon in epsilons:
        print(epsilon)
        n1d_max = int(L / epsilon)
        L = n1d_max * epsilon
        solver = ut.Solver(
            n1d_max=n1d_max,
            ndim_max=3,
            lattice_spacing=epsilon,
            derivative_shifts=get_laplace_coefficients(n_step),
        )

        E0 = zeros[0] * 2 * np.pi ** 2 / mu / L ** 2

        func = lambda c0: (solver.get_energies(c0, n_energies=1) - E0) ** 2

        c0 = -1.0

        res = opt.minimize(func, c0)
        energies = solver.get_energies(res.x, n_energies=min((n1d_max - 2) ** 2, 60))

        data.append(
            {
                "a": epsilon,
                "c": res.x[0],
                "e": energies,
                "n": n_step,
                "n1d_max": n1d_max,
            }
        )

df = pd.DataFrame(data)


In [ ]:
fig_zeta_3d_e, axs = plt.subplots(
    dpi=300, figsize=(6, 2), ncols=df.n.unique().size, sharey=True, sharex=True
)

axss = {nstep: axs[n] for n, nstep in enumerate(df.n.unique())}

for nstep, ax in axss.items():
    ax.axhline(0, ls="-", c="grey", lw=0.5, alpha=0.5)

    for div in divergencies:
        if div < S3.x.max():
            ax.axvline(div, ls="--", c="grey", lw=0.5, alpha=0.5)

    S3.plot(
        ax=ax,
        x="x",
        y="y",
        lw=0.2,
        ls=":",
        ms=0.5,
        zorder=-1,
        legend=False,
        label="$S_3$",
    )

    for zero in zeros:
        if zero < S3.x.max():
            pass  # ax.scatter(zero, 0, marker="x", c="black", lw=.5, s=5, zorder=10)

    ax.set_ylim(-5, 5)
    ax.set_xlim(-1, 6)

    ax.set_title(f"$n_{{\mathrm{{step}}}} = {nstep}$")

    ax.set_xlabel("$x$")
    ax.set_ylabel("$S_3(x)$")

markers = ["s", "o", "x", "d"]
markers = {epsilon: markers[n] for n, epsilon in enumerate(df.a.unique())}

colors = ["r", "b", "g", "y"]
colors = {epsilon: colors[n] for n, epsilon in enumerate(df.a.unique())}

for epsilon, c, energies, nstep, n1d_max in df.values:
    L = epsilon * n1d_max

    label = f"$\epsilon ={epsilon}$"

    num_zero = energies / 2 / np.pi ** 2 * mu * L ** 2
    xs = [x for x in num_zero if np.abs(x - divergencies).min() > 5.0e-2]
    ys = [[f(x)
            for (start, end), f in interpolators.items()
            if x > start and x < end
        ][0] for x in xs]
    
    xs, ys = np.transpose([[x, y] for x, y in zip(xs, ys) if np.abs(y) < 8])
    
    axss[nstep].plot(
        xs,
        ys,
        marker=markers[epsilon],
        lw=0.5,
        ls="--",
        ms=2,
        zorder=5,
        c=colors[epsilon],
        label=label,
    )


axs[0].legend(loc="upper left", fontsize=4)

plt.subplots_adjust(wspace=0.0)
plt.show(fig_zeta_3d_e)


In [ ]:
fig_zeta_3d_e.savefig("3d-zeta-fit.pdf")

In [ ]:
!open "3d-zeta-fit.pdf"

In [ ]:
energies = df.query("a == 0.05 and n == 4")["e"].values[0]
energies = np.array([e for e in energies / (2 * np.pi**2 / mu / L**2) if np.abs(e - divergencies).min() > 1.e-3 ])
energies

In [ ]:
fig, axs = plt.subplots(figsize=(6,3), dpi=200, ncols=2, sharex=True)

xnum = np.array(zeros).flatten()[:-1]

axs[0].plot(xnum, (energies -xnum), "--o")

axs[0].set_xlabel("$x$")
axs[0].set_ylabel(r"$x_{lat} - x_{num}$")

axs[1].plot(xnum, (energies -xnum)/xnum*100, "--o")

axs[1].set_xlabel("$x$")
axs[1].set_ylabel(r"$\frac{x_{lat} - x_{num}}{x_{num}}$ [%]")

plt.tight_layout()
plt.show()

In [ ]:
fig.savefig("energy-level-discrepancy.png", bbox_inches="tight")

In [ ]:
!open "energy-level-discrepancy.png"

In [ ]:
df.to_csv("luescher-3d-res.csv", index=False)